In [1]:
%matplotlib widget

In [2]:
from geopy import distance

In [17]:
import requests
import numpy as np
import matplotlib.pyplot as plt

In [4]:
laax_ids = [
    154975,
    132187,
    207482,
    132180,
    154975,
    132182,
    132181,
    132301,
    132151,
    217149,
    207520,
    132305,
    132157,
    132154,
    132155,
    207474,
    132156,
    132149,
    207475,
    189209,
    132156,
    132152,
    132148,
    207579,
    132161,
    132339,
    218343,
    132162,
    132321,
    132320,
    132168,
    207481,
    132167,
    132189,
    132182,
    132186,
    132166,
    132188,
    132165,
    132164,
    199174,
    199178,
    132287,
    132340,
    132341,
    199175,
    132160,
    132185,
    132302,
    132183,
    132421,
    207446,
    132171,
    132170,
    133043,
    132169,
    207441,
    132303,
    132198,
    132158,
    132159,
    132343,
    132172,
    132178,
    199172,
    207443,
    207480,
    168459,
    207479,
    207478,
    199173,
    132300,
    132179,
    132298,
    132174,
    132173,
    132344,
    218345,
    218344,
    217919,
    216958,
    207519
]
len(laax_ids) - len(set(laax_ids))

3

In [5]:
def calc_distance_from_geo_series(series):
    d_start = series[0]
    integral = []

    for d_stop in series[1:]:
        integral.append(distance.distance(d_start, d_stop).m)
        d_start = d_stop
    return integral

In [6]:
def get_slope_info(lid):

    slope = requests.get('https://tiles.skimap.org/features/{}.geojson'.format(lid)).json()

    headers = {
        'Content-Type': 'application/json',
    }

    data = np.array(slope['geometry']['coordinates'])[:, ::-1]
    pair_dist = calc_distance_from_geo_series(data)

    elevation = requests.post('https://elevation.racemap.com/api', headers=headers, json=data.tolist()).json()
    
    return_data = dict()
    return_data['name'] = slope['properties']['name']
    return_data['difficulty'] = slope['properties']['piste:difficulty']
    return_data['type'] = slope['properties']['piste:type']
    return_data['id'] = slope['properties']['lid']
    return_data['overall_elevation'] = elevation[0] - elevation[-1]
    return_data['distance'] = np.sum(pair_dist)
    return_data['pos'] = np.cumsum(pair_dist)
    return_data['geo'] = data
    return_data['elevation'] = elevation
    return return_data

In [178]:
slopes[132187]['geo']

array([[46.8809104,  9.2041097],
       [46.8810281,  9.2034654],
       [46.8807791,  9.202757 ],
       [46.8802793,  9.2021291],
       [46.8799958,  9.2016203],
       [46.8797738,  9.2013997],
       [46.879819 ,  9.2009836],
       [46.8800668,  9.200013 ],
       [46.880246 ,  9.1992614],
       [46.880481 ,  9.1985806],
       [46.8810775,  9.198072 ],
       [46.8811333,  9.1974756],
       [46.880958 ,  9.19692  ],
       [46.8806303,  9.1958952],
       [46.8801463,  9.1943007],
       [46.8797761,  9.1932473],
       [46.8795893,  9.1917971],
       [46.8795847,  9.1913566],
       [46.8795143,  9.1910633],
       [46.8787796,  9.1888626],
       [46.8783241,  9.1873252],
       [46.8778601,  9.185646 ],
       [46.8775546,  9.1838644],
       [46.8772024,  9.1831374],
       [46.8767052,  9.1827332],
       [46.8758698,  9.1823027],
       [46.8752514,  9.1820452],
       [46.8747073,  9.1818335],
       [46.8739744,  9.181526 ]])

In [7]:
from pathlib import Path
import pickle

path = Path('./laax_slopes.pk')
if path.is_file():
    slopes = pickle.load(path.open('rb'))
else:
    slopes = {}
    
for i in laax_ids:
    if i not in slopes:
        slopes[i] = get_slope_info(i)

pickle.dump(slopes, path.open('wb'))

In [8]:
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, TapTool

output_notebook()

Loading BokehJS ...

In [9]:
color_map = {
    'intermediate': 'red',
    'easy': 'blue',
    'freeride': 'yellow',
    'advanced': 'black'
}

c_data = dict()
c_data['id'] = list(slopes.keys())
c_data['y'] = list(g['geo'].T[0] for g in slopes.values())
c_data['x'] = list(g['geo'].T[1] for g in slopes.values())
c_data['distance'] = list(g['distance'] for g in slopes.values())
c_data['difficulty'] = list(g['difficulty'] for g in slopes.values())
c_data['height'] = list(g['elevation'] for g in slopes.values())
c_data['rel_height'] = list(np.array(g['elevation']) - g['elevation'][0] for g in slopes.values())
c_data['pos'] = list(g['pos'] for g in slopes.values())
c_data['name'] = list(g['name'] for g in slopes.values())
c_data['color'] = list(color_map[g['difficulty']] for g in slopes.values())
c_data = ColumnDataSource(c_data)

from bokeh.layouts import gridplot

TOOLTIPS = [
    ("id", "@id"),
    ("name", "@name"),
    ("distance", "@distance"),
#     ("difficulty", "@difficulty"),
#     ("height", "@height"),
    ("(x,y)", "($x, $y)"),
]

TOOLS = "box_zoom,tap,reset"

p = figure(plot_width=800, plot_height=800, tooltips=TOOLTIPS, tools=TOOLS)
p.multi_line('x', 'y', source=c_data, color='color', line_width=3)
p1 = figure(plot_width=800, plot_height=800, tooltips=TOOLTIPS, tools=TOOLS)
p1.multi_line('pos', 'rel_height', source=c_data, color='color', line_width=3)
show(gridplot([[p, p1]]))

In [241]:
slopes[207579]

{'name': 'Laax 64',
 'difficulty': 'intermediate',
 'type': 'downhill',
 'id': '207579',
 'overall_elevation': 1118.5601112062006,
 'distance': 4997.513689589484,
 'pos': array([  93.58897585,  159.20104121,  233.21958669,  292.44058786,
         355.68122281,  405.17307826,  466.12747994,  516.31226904,
         539.03116384,  567.40178694,  601.69094547,  702.76255811,
         806.69185825,  859.58664738,  991.99889931, 1049.92626143,
        1089.8786985 , 1120.29935015, 1192.93970913, 1208.24161665,
        1271.01201023, 1338.93903942, 1452.68290741, 1534.20032827,
        1611.1814805 , 1639.32388769, 1747.63303579, 1798.20887919,
        1891.64751645, 1937.33679383, 2091.27903721, 2191.53384672,
        2373.84028235, 2414.81953711, 2545.42135502, 2604.09078524,
        2660.50874972, 2694.98002387, 2724.43804249, 2775.14625392,
        2806.95736945, 2827.35654937, 2884.96064354, 2936.58907849,
        2979.43925291, 3055.85320774, 3125.80748946, 3189.19968518,
        3234.3

In [221]:
all_geo = []
for s in slopes.values():
    for p in s['geo']:
        all_geo.append((s['id'], p))

In [223]:
all_geo_points = np.vstack([p[1] for p in all_geo])

In [227]:
all_geo_points

array([[46.8765442,  9.1972722],
       [46.8739438,  9.1944747],
       [46.8725232,  9.193454 ],
       ...,
       [46.8472571,  9.2240016],
       [46.8471923,  9.2243692],
       [46.8474709,  9.22487  ]])

# Data

In [164]:
import pandas as pd

data = pd.read_csv('./laax/drive-download-20190406T125400Z-001/NilsPodX-5CF4_20190405_1344.csv', header=0)


In [165]:
def calc_height(baro, T, ref_baro, ref_height):
    R = 8.3144
    g_0 = 9.81
    M = 0.029
    T += 273
    return -np.log(baro/ref_baro) * R * T/ (g_0 * M) + ref_height
    

In [166]:
ref_baro = data[['baro']].loc[0]
#Run was started at Galaaxy
ref_height = 2250 #m
T = 0 # deg C

In [167]:
calc_height(data[['baro']], T, ref_baro, ref_height).plot()


/home/arne/anaconda3/lib/python3.6/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [168]:
top = 2563
bottom = 2173

In [169]:
acc = ['acc_' + i for i in 'xyz']
gyro = ['gyro_' + i for i in 'xyz']

In [170]:
fig, axs = plt.subplots(2, sharex=True)

data[acc].plot(ax=axs[0])
calc_height(data[['baro']], T, ref_baro, ref_height).plot(ax=axs[1])

/home/arne/anaconda3/lib/python3.6/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [171]:
import gpxpy

# Parsing an existing file:
# -------------------------

gpx_file = open('./laax/drive-download-20190406T125400Z-001/Laax_fri_Nachmittag.gpx', 'r')

gpx = gpxpy.parse(gpx_file)

In [191]:
points = gpx.tracks[0].segments[0].points
geo_track = np.array([(p.latitude, p.longitude, p.elevation, (p.time - points[0].time).total_seconds()) for p in points])

In [213]:
np.cumsum(calc_distance_from_geo_series(geo_track[:, :2]))

array([  229.67166719,   270.00861769,   295.09998449, ...,
       14697.59982511, 14700.66673576, 14704.68274065])

In [217]:
plt.figure()
plt.plot(geo_track[:,-1], geo_track[:, -2]  - geo_track[10,-2] + height['baro'].iloc[0])
heigth_sec = height.copy()
# heigth_sec.index /= 204.8
# heigth_sec.plot(ax=plt.gca())
ax2 = plt.gca().twinx()
ax2.plot(geo_track[:,-1], np.hstack([[0], np.cumsum(calc_distance_from_geo_series(geo_track[:, :2]))]))

/home/arne/anaconda3/lib/python3.6/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [23]:
plt.figure()
plt.plot(*np.array([(p.latitude, p.longitude) for p in gpx.tracks[0].segments[0].points]).T[::-1])

FigureCanvasNbAgg()

In [242]:
from numpy.linalg import norm
closest = []

for p in geo_track[:, :2]:
    diff = all_geo_points - p
    diff = norm(diff, axis=1)
    all_geo[np.argmin(diff)]
    closest.append(all_geo[np.argmin(diff)])

tracks = pd.Series([p[0] for p in closest])
difficulty = [slopes[int(t)]['difficulty'] for t in tracks]

In [245]:
difficulty

['intermediate',
 'intermediate',
 'easy',
 'easy',
 'intermediate',
 'easy',
 'easy',
 'easy',
 'easy',
 'easy',
 'easy',
 'easy',
 'easy',
 'easy',
 'easy',
 'easy',
 'easy',
 'easy',
 'easy',
 'easy',
 'easy',
 'easy',
 'easy',
 'easy',
 'easy',
 'easy',
 'easy',
 'easy',
 'easy',
 'easy',
 'easy',
 'easy',
 'easy',
 'easy',
 'easy',
 'easy',
 'easy',
 'easy',
 'easy',
 'easy',
 'intermediate',
 'intermediate',
 'intermediate',
 'intermediate',
 'intermediate',
 'intermediate',
 'intermediate',
 'easy',
 'easy',
 'easy',
 'easy',
 'easy',
 'easy',
 'intermediate',
 'intermediate',
 'intermediate',
 'intermediate',
 'intermediate',
 'intermediate',
 'intermediate',
 'intermediate',
 'intermediate',
 'intermediate',
 'intermediate',
 'intermediate',
 'intermediate',
 'intermediate',
 'intermediate',
 'intermediate',
 'intermediate',
 'intermediate',
 'intermediate',
 'intermediate',
 'intermediate',
 'intermediate',
 'intermediate',
 'intermediate',
 'intermediate',
 'intermediate',
 

In [244]:
plt.figure()
pd.Series(difficulty).plot()

/home/arne/anaconda3/lib/python3.6/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

TypeError: Empty 'DataFrame': no numeric data to plot

In [236]:
plt.figure()
tracks.plot()

/home/arne/anaconda3/lib/python3.6/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [226]:
closest

[('154975', array([46.8765442,  9.1972722])),
 ('154975', array([46.8765442,  9.1972722])),
 ('154975', array([46.8765442,  9.1972722])),
 ('154975', array([46.8765442,  9.1972722])),
 ('154975', array([46.8765442,  9.1972722])),
 ('154975', array([46.8765442,  9.1972722])),
 ('154975', array([46.8765442,  9.1972722])),
 ('154975', array([46.8765442,  9.1972722])),
 ('154975', array([46.8765442,  9.1972722])),
 ('154975', array([46.8765442,  9.1972722])),
 ('154975', array([46.8765442,  9.1972722])),
 ('154975', array([46.8765442,  9.1972722])),
 ('154975', array([46.8765442,  9.1972722])),
 ('154975', array([46.8765442,  9.1972722])),
 ('154975', array([46.8765442,  9.1972722])),
 ('154975', array([46.8765442,  9.1972722])),
 ('154975', array([46.8765442,  9.1972722])),
 ('154975', array([46.8765442,  9.1972722])),
 ('154975', array([46.8765442,  9.1972722])),
 ('154975', array([46.8765442,  9.1972722])),
 ('154975', array([46.8765442,  9.1972722])),
 ('154975', array([46.8765442,  9.

In [24]:
from scipy.signal import butter, filtfilt

def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = filtfilt(b, a, data)
    return y

In [25]:
data[acc].apply(lambda x: butter_lowpass_filter(x, 3, 204.8)).plot()

/home/arne/anaconda3/lib/python3.6/site-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


FigureCanvasNbAgg()

In [199]:
fig, axs = plt.subplots(3, sharex=True, figsize=(8, 10))

data[acc].rolling(600).median().plot(ax=axs[0])
gyro_filtered = data[gyro].rolling(600).mean()
gyro_filtered.plot(ax=axs[1])
height = calc_height(data[['baro']], T, ref_baro, ref_height)
calc_height(data[['baro']], T, ref_baro, ref_height).plot(ax=axs[2])

plt.tight_layout()

/home/arne/anaconda3/lib/python3.6/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [155]:
from scipy.signal import savgol_filter

def baro_segment(height_data):
    filtered = height_data.rolling(1000).median().diff().rolling(2000).sum()
    segments = pd.Series(np.array([0] * len(height)))
    segments.index = filtered.index
    segments[(filtered > 1).values] = 1
    segments[(filtered < -2).values] = -1
    return segments

In [ ]:
from bokeh.models import BoxAnnotation

p = figure(plot_width=800, plot_height=800)
p.line(height)
BoxAnnotation

In [156]:
plt.figure()
plt.plot(baro_segment(height['baro']))
((height/1000) - 1.5).plot(ax=plt.gca())

/home/arne/anaconda3/lib/python3.6/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [108]:
def find_active_regions(data):
    # Must be moving downwards:
    return (height.rolling(1000).median().diff().rolling(2000).sum() < -1).fillna(False)

In [109]:
gyro_active_filtered = gyro_filtered[find_active_regions(data).values]

In [110]:
gyro_active_filtered.plot()

/home/arne/anaconda3/lib/python3.6/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

In [60]:
fig, axs = plt.subplots(3, sharex=True, figsize=(8, 10))

data_active[acc].rolling(600).median().plot(ax=axs[0])
data_active[gyro].rolling(600).median().plot(ax=axs[1])
calc_height(data_active[['baro']], T, ref_baro, ref_height).plot(ax=axs[2])

plt.tight_layout()

FigureCanvasNbAgg()

TypeError: Empty 'DataFrame': no numeric data to plot

In [54]:
d = np.where(np.diff(np.sign(data['gyro_z'].rolling(600).median())))[0]
plt.figure()
plt.plot(d, [0]*len(d), 'o') 

FigureCanvasNbAgg()

In [27]:
from slither.imu.orientation import MadgewickAHRS

In [42]:
ori = MadgewickAHRS(sampling_rate=204.8)
ori = ori.update_series(np.deg2rad((data[gyro] - data[gyro].iloc[:2000].mean(axis=0)).values), data[acc].values)

In [43]:
plt.figure()
plt.plot(ori)

FigureCanvasNbAgg()